In [18]:
#imports
import pulp as p
import numpy as np
import matplotlib as plt

In [19]:
#fixed values

#hier input fenster bauen, sodass es benutzerfreundlicher wird

# Muss aus Strompreis/COP berechnet werden
c = [
    0.0556, 0.0522, 0.0489, 0.0458, 0.0429, 0.0400, 
    0.0458, 0.0565, 0.0682, 0.0833, 0.0829, 0.0825, 
    0.0780, 0.0698, 0.0622, 0.0587, 0.0659, 0.0810, 
    0.0854, 0.0786, 0.0698, 0.0622, 0.0565, 0.0532
]

T = len(c)
p_max = 5
s_max = 7
s_laden = 3
s_entladen = 3
d = 2.8
B_start = 0

In [ ]:
#Hier Grafik und Herleitung für: Von Wetter und Wärmebedarf zu COP, von COP und Strompreis zu C  

In [ ]:
#Hier grafische Erklärung für die Nebenbedingungen

# Zielfunktion und Nebenbedingungen für das Optimierungsproblem

## Zielfunktion
Minimierung der Gesamtkosten:
- \( P_t \): Wärmeerzeugung der Wärmepumpe in Stunde \( t \)
- \( c_t \): Kosteneffizienz (EUR pro kW Wärme) in Stunde \( t \)
- Ziel: Minimierung der Gesamtkosten über alle Stunden

\[
\min \sum_{t=0}^{23} c_t \cdot P_t
\]

---

## Nebenbedingungen

1. **Grenzen der Wärmeerzeugung**
   - Die Wärmeerzeugung \( P_t \) ist durch die maximale Leistung der Wärmepumpe beschränkt:
   \[
   0 \leq P_t \leq P_{\text{max}}
   \]

2. **Grenzen für das Laden und Entladen des Speichers**
   - Einspeicherung (\( x^{\text{in}}_t \)) und Ausspeicherung (\( x^{\text{out}}_t \)) des Speichers sind beschränkt:
   \[
   0 \leq x^{\text{in}}_t \leq X_{\text{in,max}}, \quad 0 \leq x^{\text{out}}_t \leq X_{\text{out,max}}
   \]

3. **Grenzen für den Speicherfüllstand**
   - Der Speicherfüllstand (\( B_t \)) ist durch die Speicherkapazität begrenzt:
   \[
   0 \leq B_t \leq B_{\text{max}}
   \]

4. **Wärmebilanz**
   - Die Wärmenachfrage \( d \) muss durch die Wärmeerzeugung \( P_t \), Einspeicherung (\( x^{\text{in}}_t \)), und Ausspeicherung (\( x^{\text{out}}_t \)) gedeckt werden:
   \[
   P_t + x^{\text{out}}_t = d + x^{\text{in}}_t, \quad \forall t \in \{0, \dots, 23\}
   \]

5. **Dynamik des Speicherfüllstands**
   - Der Speicherfüllstand \( B_t \) entwickelt sich über die Zeit durch Einspeicherung (\( x^{\text{in}}_t \)) und Ausspeicherung (\( x^{\text{out}}_t \)):
   \[
   B_0 = B_{\text{start}} + x^{\text{in}}_0 - x^{\text{out}}_0
   \]
   \[
   B_t = B_{t-1} + x^{\text{in}}_t - x^{\text{out}}_t, \quad \forall t \in \{1, \dots, 23\}
   \]

---

## Notation
- \( P_t \): Wärmeerzeugung (kW)
- \( x^{\text{in}}_t \): Einspeicherung (kW)
- \( x^{\text{out}}_t \): Ausspeicherung (kW)
- \( B_t \): Speicherfüllstand (kWh)
- \( c_t \): Kosteneffizienz (EUR/kW)
- \( d \): Wärmenachfrage (kW)
- \( P_{\text{max}} \): Maximale Wärmeerzeugungskapazität (kW)
- \( X_{\text{in,max}}, X_{\text{out,max}} \): Maximale Lade- und Entladeleistung (kW)
- \( B_{\text{max}} \): Maximale Speicherkapazität (kWh)
- \( B_{\text{start}} \): Anfangsfüllstand des Speichers (kWh)


In [20]:
model = p.LpProblem("problem", p.LpMinimize)

In [23]:
#decision variables
P = [p.LpVariable(f"p_{t}", lowBound=0, upBound=p_max) for t in range(T)]
B = [p.LpVariable(f"B_{t}", lowBound=0, upBound=s_max) for t in range(T)]
x_in = [p.LpVariable(f"x_in_{t}", lowBound=0, upBound=s_laden) for t in range(T)]
x_out = [p.LpVariable(f"x_out_{t}", lowBound=0, upBound=s_entladen) for t in range(T)]

In [33]:
#objective function
model += p.lpSum(c[t] * P[t] for t in range(T))

In [48]:
#constraints
#Grenzen für den Füllstand des Pufferspeichers
for t in range(T):
    model += B[t] >= 0  # Untergrenze Füllstand
    model += B[t] <= 7  # Obergrenze Füllstand

#Wärmebilanz
for t in range(T):
    model += P[t] + x_out[t] == d + x_in[t]
    
#Speicherzustand Dynamik
model += B[0] == B_start * x_in[0] - x_out[0]  # Startfüllstand

for t in range(1, T):
    model += B[t] == B[t-1] + x_in[t] - x_out[t]  # Dynamik
#noch n ergänzen, also Wärmeverlustquote


In [54]:
status = model.solve() 
print(p.LpStatus[status])


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/juliandomnik/PycharmProjects/heatpump-operation-optimization/.venv/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/m4/qcxwdmxd5b94sx7ctbs447dc0000gn/T/bae6fb39807846f790bc0437f7d25f61-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/m4/qcxwdmxd5b94sx7ctbs447dc0000gn/T/bae6fb39807846f790bc0437f7d25f61-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 197 COLUMNS
At line 650 RHS
At line 843 BOUNDS
At line 940 ENDATA
Problem MODEL has 192 rows, 96 columns and 428 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 46 (-146) rows, 91 (-5) columns and 159 (-269) elements
Perturbing problem by 0.001% of 0.0854 - largest nonzero change 9.8850144e-07 ( 0.0022025455%) - largest zero change 9.8644537e-07
0  Obj 0.15568 Primal inf 64.399977 (23)
42  Obj 2.9685584 Prim

In [55]:
#optimal values

print("Optimale Ergebnisse:")
for t in range(24):
    print(
        f"Stunde {t}: "
        f"P[t] = {P[t].varValue:.3f}, "
        f"x_in[t] = {x_in[t].varValue:.3f}, "
        f"x_out[t] = {x_out[t].varValue:.3f}, "
        f"B[t] = {B[t].varValue:.3f}"
    )


Optimale Ergebnisse:
Stunde 0: P[t] = 2.800, x_in[t] = 0.000, x_out[t] = 0.000, B[t] = 0.000
Stunde 1: P[t] = 2.800, x_in[t] = 0.000, x_out[t] = 0.000, B[t] = 0.000
Stunde 2: P[t] = 2.800, x_in[t] = 0.000, x_out[t] = 0.000, B[t] = 0.000
Stunde 3: P[t] = 3.200, x_in[t] = 0.400, x_out[t] = 0.000, B[t] = 0.400
Stunde 4: P[t] = 5.000, x_in[t] = 2.200, x_out[t] = 0.000, B[t] = 2.600
Stunde 5: P[t] = 5.000, x_in[t] = 2.200, x_out[t] = 0.000, B[t] = 4.800
Stunde 6: P[t] = 5.000, x_in[t] = 2.200, x_out[t] = 0.000, B[t] = 7.000
Stunde 7: P[t] = 2.800, x_in[t] = 0.000, x_out[t] = 0.000, B[t] = 7.000
Stunde 8: P[t] = 2.800, x_in[t] = 0.000, x_out[t] = 0.000, B[t] = 7.000
Stunde 9: P[t] = 0.000, x_in[t] = 0.000, x_out[t] = 2.800, B[t] = 4.200
Stunde 10: P[t] = 0.000, x_in[t] = 0.000, x_out[t] = 2.800, B[t] = 1.400
Stunde 11: P[t] = 1.400, x_in[t] = 0.000, x_out[t] = 1.400, B[t] = 0.000
Stunde 12: P[t] = 2.800, x_in[t] = 0.000, x_out[t] = 0.000, B[t] = 0.000
Stunde 13: P[t] = 3.200, x_in[t] = 0.400

In [ ]:
#Hier Grafik für optimalen Betrieb (plt)

In [57]:
#Mini-Zusatzlektion Lineare Algebra / Veranschaulichung des Problems 
print("Modelldimensionen: " + str(len(model.constraints)) + " x "+ str(len(model.variables())))
print("------------------")
print("Anzahl der Constraints (m):", len(model.constraints))
print("Anzahl der Variablen (n):", len(model.variables()))


Modelldimensionen: 192 x 96
------------------
Anzahl der Constraints (m): 192
Anzahl der Variablen (n): 96


In [ ]:
#Amortisationsdauer

